In [82]:
'''
compare two implementations of price oracle:
    arithmetic mean used by uniswap v2
    geometric mean used by uniswap v3
'''

from functools import reduce
import math
import pandas as pd
import numpy as np


In [83]:
# tick index is log(price, 1.0001)
LOG_BASE = 1.0001
time_index = [0,1,3,4,5,9,12,13,16,20,22]
# there is a hostile price on 5th element (price 30)
origin_prices = [100,130,110,100,30,100,110,120,110,120,100]

# delta time of every besides price
# first element is 0s
delta_times = [0]
for i in range(1, len(time_index)):
    delta_times.append(time_index[i] - time_index[i-1])
    

In [84]:
# uniswap V2 use arithmetic sum
def arithmetic_sums_calc(begin, end):
    sums = []
    for i in range(begin, end):
        if i == 0:
            sums.append(origin_prices[0])
        else:
            # sum = current_price * delta time
            sums.append(origin_prices[begin+i] * delta_times[begin+i])
    return sums

# uniswap V3 use geometric sum
def geometric_sums_calc(begin, end):
    sums = []
    for i in range(begin, end):
        if i == 0:
            sums.append(math.log(origin_prices[0], LOG_BASE))
        else:
            # sum = current_price * delta time
            sums.append(math.log(origin_prices[begin+i], LOG_BASE) * delta_times[begin+i])
    return sums

arithmetic_sums = arithmetic_sums_calc(0, len(origin_prices))
geometric_sums = geometric_sums_calc(0, len(origin_prices))

print(arithmetic_sums)
print(geometric_sums)

[100, 130, 220, 100, 30, 400, 330, 120, 330, 480, 200]
[46054.00440660449, 48677.77823122565, 94014.30771788706, 46054.00440660449, 34013.674386974235, 184216.01762641795, 141021.4615768306, 47877.311133803356, 141021.4615768306, 191509.24453521342, 92108.00881320897]


In [85]:
# the cumulative price store in oracle
cumulative_arithmetic = []
cumulative_geometric = []

for i in range(1,len(origin_prices)+1):
    cumulative_arithmetic.append(reduce(lambda x,y: x+y, arithmetic_sums[0:i]))
    cumulative_geometric.append(reduce(lambda x,y: x+y, geometric_sums[0:i]))

print(cumulative_arithmetic)
print(cumulative_geometric)


[100, 230, 450, 550, 580, 980, 1310, 1430, 1760, 2240, 2440]
[46054.00440660449, 94731.78263783014, 188746.0903557172, 234800.09476232168, 268813.7691492959, 453029.7867757139, 594051.2483525445, 641928.5594863478, 782950.0210631784, 974459.2655983919, 1066567.2744116008]


In [86]:
# calc price with oracle's cumulative price
def calc_arithmetic_oracleprice(begin,end):
    return (cumulative_arithmetic[end] - cumulative_arithmetic[begin]) / (time_index[end] - time_index[begin])

def calc_geometric_oracleprice(begin,end):
    return math.pow(LOG_BASE, (cumulative_geometric[end] - cumulative_geometric[begin]) / (time_index[end] - time_index[begin]))

WINDOW_SIZE = 4

np_mean_prices = []
oracle_arithmetic_prices = []
oracle_geometric_prices = []

for i in range(0,len(origin_prices)):
    if i < WINDOW_SIZE:
        np_mean_prices.append(None)
        oracle_arithmetic_prices.append(None)
        oracle_geometric_prices.append(None)
    else:
        np_mean_prices.append(np.mean(origin_prices[i-WINDOW_SIZE:i]))
        oracle_arithmetic_prices.append(calc_arithmetic_oracleprice(i-WINDOW_SIZE,i))
        oracle_geometric_prices.append(calc_geometric_oracleprice(i-WINDOW_SIZE,i))

df = pd.DataFrame({
  'time_index': time_index,
  'price': origin_prices,
  'delta_times': delta_times,
  'cumulative_arithmetic': cumulative_arithmetic,
  'cumulative_geometric': cumulative_geometric,
  'oracle_arithmetic_prices': oracle_arithmetic_prices,
  'oracle_geometric_prices': oracle_geometric_prices,
})

# the result showing arithmetic mean is more stable than geometric mean
# that's weird...
print(df)


    time_index  price  delta_times  cumulative_arithmetic  \
0            0    100            0                    100   
1            1    130            1                    230   
2            3    110            2                    450   
3            4    100            1                    550   
4            5     30            1                    580   
5            9    100            4                    980   
6           12    110            3                   1310   
7           13    120            1                   1430   
8           16    110            3                   1760   
9           20    120            4                   2240   
10          22    100            2                   2440   

    cumulative_geometric  oracle_arithmetic_prices  oracle_geometric_prices  
0           4.605400e+04                       NaN                      NaN  
1           9.473178e+04                       NaN                      NaN  
2           1.887461e+05         